In [47]:
import numpy as np
import sympy as sp
import matplotlib.pyplot as plt
import math     

In [48]:
def calculate_der(n,point):
    x = sp.symbols('x')
    f = sp.sin(1000*x) / (1 + x)
    df = sp.diff(f, x)
    if n == 1:
        return df.evalf(subs={x: point})
    for i in range(n-1):
        df = sp.diff(df,x)

    return df.evalf(subs={x: point})


In [49]:
def max_der(a,b,n):
    x = np.linspace(a,b,10**(3))
    y = []
    for i in x:
        y.append(calculate_der(n,i))
    return max(y)

# Вычисление нужного числа отрезков для достижения точности

In [50]:
def n_Trapezoidal(a,b,eps):
    M = max_der(a,b,2)
    C = 12
    h = math.sqrt(eps * C/ ((b-a)*M))
    n = int((b-a)/h)
    return n 
n = n_Trapezoidal(0,1,10**(-6))
print(n)

287077


In [51]:
def n_Simpson(a,b,eps):
    M = max_der(a,b,4)
    C = 2880
    h = (eps * C/ ((b-a)*M))**(1/4)
    n = int((b-a)/h)
    return n 
n = n_Simpson(0,1,10**(-6))
print(n)

4294


In [52]:
def n_3_8(a,b,eps):
    M = max_der(a,b,4)
    C = 6480
    h = (eps * C/ ((b-a)*M))**(1/4)
    n = int((b-a)/h)
    return n 
n = n_3_8(0,1,10**(-6))
print(n)

3506


# Правило трапеции

In [53]:
n = n_Trapezoidal(0,1,10**(-6))
def Trapezoidal_rule(f,a,b,n):
    S = 0
    x = np.linspace(a,b,n)
    for i in range(1,len(x)):
        S += (f(x[i-1]) + f(x[i]))/2 * (x[i]-x[i-1])
    return S

f = lambda x: math.sin(1000*x) / (1 + x)
I_trap = Trapezoidal_rule(f,0,1,n)
print(I_trap)


0.0007186011558477808


# Правило Симпсона

In [54]:
n = n_Simpson(0,1,10**(-6))
def Simpson_rule(f,a,b,n):
    S = 0
    x = np.linspace(a,b,n)
    for i in range(1,len(x)):
        S += (f(x[i-1]) + 4 * f((x[i-1] + x[i])/2) + f(x[i]))/6 * (x[i]-x[i-1])
    return S

f = lambda x: math.sin(1000*x) / (1 + x)
I_simpson = Simpson_rule(f,0,1,n)
print(I_simpson)

0.0007186026195317587


# Правило 3/8

In [55]:
n = n_3_8(0,1,10**(-6))
def three_eight_rule(f,a,b,n):
    S = 0
    x = np.linspace(a,b,n)
    for i in range(1,len(x)):
        S += (f(x[i-1]) + 3 * f((2*x[i-1] + x[i])/3) + 3*f((x[i-1]+2*x[i])/3)+f(x[i]))/8 * (x[i]-x[i-1])
    return S

f = lambda x: math.sin(1000*x) / (1 + x)
I_3_8 = three_eight_rule(f,0,1,n)
print(I_3_8)

0.0007186026201146368


# Метод Гаусса

In [70]:
from sympy import symbols 
from sympy import solve
import math
def Gauss_integration(f, a, b, n):
    f1 = lambda x: - math.cos(1000*x)/1000
    f2 = lambda x: (math.sin(1000*x)-1000*x*math.cos(1000*x))/10**6
    f3 = lambda x: (1000*x*math.sin(1000*x)+(1-500000*x**2)*math.cos(1000*x))/(5*10**8)
    f4 = lambda x: ((1500000*x**2-3)*math.sin(1000*x)+(3000*x-500000000*x**3)*math.cos(1000*x))/(5*10**11)
    integral1 = f1(1)-f1(0)
    integral2 = f2(1)-f2(0)
    integral3 = f3(1)-f3(0)
    integral4 = f4(1)-f4(0)
    c1, c2, x1, x2 = symbols("c1, c2, x1, x2")
    equations = [c1 + c2 - integral1, c1*x1 + c2*x2 - integral2, c1*x1**2 + c2*x2**2 - integral3, c1*x1**3 + c2*x2**3 - integral4]
    solutions = solve(equations, [x1,x2,c1,c2], dict=True)
    C1 = solutions[0][c1]; C12 = solutions[1][c1]
    C2 = solutions[0][c2]; C22 = solutions[1][c2]
    X1 = solutions[0][x1]; X12  = solutions[1][x1]
    X2 = solutions[0][x2]; X22 = solutions[1][x2]
    I1 = C1*1/(1+X1) + C2*1/(1+X2)
    I2 = C12*1/(1+X12) + C22*1/(1+X22)
    print("Weights: ", C1, C2)
    print("Nodes: ", X1, X2)
    print("Integral: ", I1)
    # print("\n")
    # print("Weights: ", C12, C22)
    # print("Nodes: ", X12, X22)
    # print("Integral: ",I2)
    return I1, I2

In [71]:
f = lambda x: math.sin(1000*x) / (1 + x)
I1, I2 = Gauss_integration(f, 0, 1, 2)

Weights:  -0.000562392143827906 0.00100001306753720
Nodes:  0.998517771869666 6.35353884913004e-6
Integral:  0.000718602089116272


In [69]:
print("{:.4e}".format(I1-I_trap))
print("{:.4e}".format(I1-I_simpson))
print("{:.4e}".format(I1-I_3_8))

9.3327e-10
-5.3042e-10
-5.3100e-10
